In [ ]:
# Import scraping tools load initial page 
from bs4 import BeautifulSoup
import requests

url = "https://opencircuit.nl/list/arduino"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

# print(soup)

In [ ]:
# Get table that holds product information 
table = soup.find("div", {'class':'CategoryData' })
if table: 
    li_tags = table.find_all('li')
else:
    li_tags = [] 

# Print frist 3 items to inspects structure 
for i in range(min(3, len(li_tags))):
    print(li_tags[i].text.strip(), "\n" + "-"*80)

In [ ]:
import re
# Store extracted data 
products = [] 

# Parsing function 
for li in li_tags:
    text = li.text.strip()

    # Extract product name 
    name_match = re.match(r"^(.*?)-", text)
    name = name_match.group(1).strip() if name_match else text.split('\n')[0].strip()

    # Extract price 
    price_match = re.search(r"€\s?(\d+,\d+)", text)
    price = price_match.group(0) if price_match else "N/A"

    # Extract availability 
    if "Direct leverbaar" in text:
        availability = "In stock"
    elif "Niet op voorraad" in text:
        availability = "Out of stock"
    else:
        availability = "Unknown "

    
    # Extract description (everything between name and price)
    desc_start = text.find(name) + len(name)
    desc_end = text.find(price) if price != "N/A" else len(text)
    description = text[desc_start:desc_end].strip(" -\n")

    # Append to product list 
    products.append({"Name": name, "Description": description, "Price":price, "Availability": availability})

    



In [ ]:
# Convert to DataFrame
import pandas as pd 

df = pd.DataFrame(products)
df.head() # Show first few rows 

In [ ]:
# Save Dat to CSV for you BOM 
df.to_csv("bom_prices.csv", index=False)
print("Saved to 'bom_prices.csv' :) ")

In [ ]:
# show DataFrame
display(df)